##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Validación de datos mediante canalización de TFX y TensorFlow Data Validation

Nota: Recomendamos ejecutar este tutorial en un bloc de notas de Colab, ¡no es necesario configurarlo! Simplemente haga clic en "Ejecutar en Google Colab".

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tfdv"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/penguin_tfdv.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
<td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/penguin_tfdv.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tfx/tutorials/tfx/penguin_tfdv.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table></div>

En este tutorial basado en un bloc de notas, crearemos y ejecutaremos canalizaciones de TFX para validar los datos de entrada y crear un modelo de ML. Este bloc de notas se basa en la canalización de TFX que creamos en el [Tutorial de canalizaciones simples de TFX](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple). Si aún no ha leído ese tutorial, debe leerlo antes de continuar con este bloc de notas.

La primera tarea en cualquier proyecto de ciencia de datos o ML es comprender y limpiar los datos, lo que incluye lo siguiente:

- Comprender los tipos de datos, las distribuciones y otra información (p. ej., valor medio o número de valores únicos) sobre cada característica.
- Generar un esquema preliminar que describa los datos.
- Identificar anomalías y valores faltantes en los datos con respecto a un esquema dado.

En este tutorial, crearemos dos canalizaciones de TFX.

Primero, crearemos una canalización para analizar el conjunto de datos y generar un esquema preliminar del conjunto de datos dado. Este proceso incluirá dos nuevos componentes, `StatisticsGen` y `SchemaGen`.

Una vez que tengamos un esquema adecuado de los datos, crearemos una canalización para entrenar un modelo de clasificación de ML basado en la canalización del tutorial anterior. En este canal, usaremos el esquema del primer canal y un nuevo componente, `ExampleValidator`, para validar los datos de entrada.

Los tres nuevos componentes, StatisticsGen, SchemaGen y ExampleValidator, son componentes de TFX que se usan para análisis y validación de datos y se implementan a través de la biblioteca [TensorFlow Data Validation](https://www.tensorflow.org/tfx/guide/tfdv).

Consulte [Explicación de las canalizaciones de TFX](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines) para obtener más información sobre varios conceptos en TFX.

## Preparación

Primero tenemos que instalar el paquete de Python para TFX y descargar el conjunto de datos que usaremos para nuestro modelo.

### Actualización de pip

Para evitar actualizar Pip en un sistema cuando se ejecuta localmente, verifique que se esté ejecutando en Colab. Por supuesto, los sistemas locales se pueden actualizar por separado.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### Instalación de TFX


In [ ]:
!pip install -U tfx

### Desinstalación de shapely

TODO(b/263441833) Esta es una solución temporal para evitar un ImportError. En última instancia, debería solucionarse admitiendo una versión reciente de Bigquery, en lugar de desinstalar otras dependencias adicionales.

In [ ]:
!pip uninstall shapely -y

### ¿Reinició el tiempo de ejecución?

Si está usando Google Colab, la primera vez que ejecute la celda anterior, debe hacer clic en el botón "REINICIAR TIEMPO DE EJECUCIÓN" o usar el menú "Tiempo de ejecución &gt; Reiniciar tiempo de ejecución ..." para reiniciar el tiempo de ejecución. Esto se debe a la forma en que Colab carga los paquetes.

Verifique las versiones de TensorFlow y TFX.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

### Configuración de variables

Hay algunas variables que se utilizan para definir una canalización. Puede personalizar estas variables como desee. De forma predeterminada, todas las salidas de la canalización se generarán en el directorio actual.

In [ ]:
import os

# We will create two pipelines. One for schema generation and one for training.
SCHEMA_PIPELINE_NAME = "penguin-tfdv-schema"
PIPELINE_NAME = "penguin-tfdv"

# Output directory to store artifacts generated from the pipeline.
SCHEMA_PIPELINE_ROOT = os.path.join('pipelines', SCHEMA_PIPELINE_NAME)
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
SCHEMA_METADATA_PATH = os.path.join('metadata', SCHEMA_PIPELINE_NAME,
                                    'metadata.db')
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### Preparación de datos de ejemplo

Descargaremos el conjunto de datos de ejemplo para usarlo en nuestra canalización de TFX. El conjunto de datos que usamos es el [conjunto de datos Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/articles/intro.html), que también se usa en otros [ejemplos de TFX](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin).

Este conjunto de datos tiene cuatro características numéricas:

- culmen_length_mm
- culmen_depth_mm
- flipper_length_mm
- body_mass_g

Todas las características ya estaban normalizadas en un rango [0,1]. Compilaremos un modelo de clasificación que prediga las `species` de pingüinos.

Debido a que TFX ExampleGen lee entradas de un directorio, tenemos que crear un directorio y copiar el conjunto de datos en él.

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

Veamos rápidamente al archivo CSV.

In [ ]:
!head {_data_filepath}

Debería poder ver cinco columnas de características. `species` es una de 0, 1 o 2, y todas las demás características deben tener valores entre 0 y 1. Crearemos una canalización de TFX para analizar este conjunto de datos.

## Generación de un esquema preliminar

Las canalizaciones de TFX se definen mediante las API de Python. Crearemos una canalización para generar un esquema a partir de los ejemplos de entrada automáticamente. Puede revisar este esquema y ajustarlo para que se adapte a sus necesidades. Una vez finalizado el esquema, se puede usar para entrenamiento y validación de ejemplos en tareas posteriores.

Además de `CsvExampleGen` que se usa en el [Tutorial de canalizaciones simples de TFX](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple), usaremos `StatisticsGen` y `SchemaGen`:

- [StatisticsGen](https://www.tensorflow.org/tfx/guide/statsgen) calcula estadísticas para el conjunto de datos.
- [SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen) examina las estadísticas y crea un esquema de datos inicial.

Consulte las guías de cada componente o el [tutorial de componentes de TFX](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras) para obtener más información sobre estos componentes.

### Cómo escribir una definición de canalización

Definimos una función para crear una canalización de TFX. Un objeto `Pipeline` representa una canalización de TFX que se puede ejecutar con uno de los sistemas de orquestación de canalizaciones que admite TFX.

In [ ]:
def _create_schema_pipeline(pipeline_name: str,
                            pipeline_root: str,
                            data_root: str,
                            metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a pipeline for schema generation."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # NEW: Computes statistics over data for visualization and schema generation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # NEW: Generates schema based on the generated statistics.
  schema_gen = tfx.components.SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)

  components = [
      example_gen,
      statistics_gen,
      schema_gen,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

### Cómo ejecutar la canalización

Usaremos `LocalDagRunner` como en el tutorial anterior.

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_schema_pipeline(
      pipeline_name=SCHEMA_PIPELINE_NAME,
      pipeline_root=SCHEMA_PIPELINE_ROOT,
      data_root=DATA_ROOT,
      metadata_path=SCHEMA_METADATA_PATH))

Debería ver el mensaje "INFO:absl:Component SchemaGen is finished" (INFO:absl:Component SchemaGen finalizó) si la canalización se completó correctamente.

Examinaremos la salida de la canalización para comprender el conjunto de datos.

### Revisión de la salida de la canalización

Como se explicó en el tutorial anterior, una canalización de TFX produce dos tipos de salidas, artefactos y una [base de datos de metadatos (MLMD)](https://www.tensorflow.org/tfx/guide/mlmd) que contiene metadatos de artefactos y ejecuciones de canalizaciones. Definimos la ubicación de estas salidas en las celdas anteriores. De forma predeterminada, los artefactos se almacenan en el directorio `pipelines` y los metadatos se almacenan como una base de datos sqlite en el directorio `metadata`.

Puede utilizar las API de MLMD para localizar estas salidas mediante programación. Primero, definiremos algunas funciones de utilidad para buscar artefactos de salida que se acaban de producir.


In [ ]:
from ml_metadata.proto import metadata_store_pb2
# Non-public APIs, just for showcase.
from tfx.orchestration.portable.mlmd import execution_lib

# TODO(b/171447278): Move these functions into the TFX library.

def get_latest_artifacts(metadata, pipeline_name, component_id):
  """Output artifacts of the latest run of the component."""
  context = metadata.store.get_context_by_type_and_name(
      'node', f'{pipeline_name}.{component_id}')
  executions = metadata.store.get_executions_by_context(context.id)
  latest_execution = max(executions,
                         key=lambda e:e.last_update_time_since_epoch)
  return execution_lib.get_output_artifacts(metadata, latest_execution.id)

# Non-public APIs, just for showcase.
from tfx.orchestration.experimental.interactive import visualizations

def visualize_artifacts(artifacts):
  """Visualizes artifacts using standard visualization modules."""
  for artifact in artifacts:
    visualization = visualizations.get_registry().get_visualization(
        artifact.type_name)
    if visualization:
      visualization.display(artifact)

from tfx.orchestration.experimental.interactive import standard_visualizations
standard_visualizations.register_standard_visualizations()

Ahora podemos examinar las salidas de la ejecución de la canalización.

In [ ]:
# Non-public APIs, just for showcase.
from tfx.orchestration.metadata import Metadata
from tfx.types import standard_component_specs

metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    SCHEMA_METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:
  # Find output artifacts from MLMD.
  stat_gen_output = get_latest_artifacts(metadata_handler, SCHEMA_PIPELINE_NAME,
                                         'StatisticsGen')
  stats_artifacts = stat_gen_output[standard_component_specs.STATISTICS_KEY]

  schema_gen_output = get_latest_artifacts(metadata_handler,
                                           SCHEMA_PIPELINE_NAME, 'SchemaGen')
  schema_artifacts = schema_gen_output[standard_component_specs.SCHEMA_KEY]

Es hora de examinar las salidas de cada componente. Como se describió anteriormente, [Tensorflow Data Validation (TFDV)](https://www.tensorflow.org/tfx/data_validation/get_started) se usa en `StatisticsGen` y `SchemaGen`, y TFDV también proporciona visualización de las salidas de estos componentes.

En este tutorial, usaremos los métodos ayudantes de visualización en TFX que usan TFDV internamente para mostrar la visualización.

#### Análisis de la salida de StatisticsGen


In [ ]:
# docs-infra: no-execute
visualize_artifacts(stats_artifacts)

<!-- <img class="tfo-display-only-on-site"
src="images/penguin_tfdv/penguin_tfdv_statistics.png"/> -->

Puede ver varias estadísticas para los datos de entrada. Estas estadísticas se proporcionan a `SchemaGen` para construir un esquema inicial de datos automáticamente.


#### Análisis de la salida de SchemaGen


In [ ]:
visualize_artifacts(schema_artifacts)

Este esquema se infiere automáticamente a partir de la salida de StatisticsGen. Debería poder ver 4 características FLOAT y 1 característica INT.

### Exportación del esquema para uso futuro

Tenemos que revisar y refinar el esquema generado. El esquema revisado debe persistir para que podamos usarlo en procesos posteriores para el entrenamiento del modelo de ML. En otras palabras, quizás convenga agregar el archivo de esquema al sistema de control de versiones para casos de uso reales. En este tutorial, simplemente copiaremos el esquema a una ruta del sistema de archivos predefinida para simplificar.


In [ ]:
import shutil

_schema_filename = 'schema.pbtxt'
SCHEMA_PATH = 'schema'

os.makedirs(SCHEMA_PATH, exist_ok=True)
_generated_path = os.path.join(schema_artifacts[0].uri, _schema_filename)

# Copy the 'schema.pbtxt' file from the artifact uri to a predefined path.
shutil.copy(_generated_path, SCHEMA_PATH)

El archivo de esquema usa el [formato de texto Protocol Buffer](https://googleapis.dev/python/protobuf/latest/google/protobuf/text_format.html) y una instancia del [protocolo TensorFlow Metadata Schema](https://github.com/tensorflow/metadata/blob/master/tensorflow_metadata/proto/v0/schema.proto).

In [ ]:
print(f'Schema at {SCHEMA_PATH}-----')
!cat {SCHEMA_PATH}/*

Debe asegurarse de revisar y posiblemente editar la definición del esquema según sea necesario. En este tutorial, usaremos el esquema generado sin cambios.


## Validación de ejemplos de entrada y entrenamiento un modelo de ML

Volveremos a la canalización que creamos en el [Tutorial de canalizaciones simples de TFX](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple) para entrenar un modelo de ML y usar el esquema generado para escribir el código de entrenamiento del modelo.

También agregaremos un componente [ExampleValidator](https://www.tensorflow.org/tfx/guide/exampleval) que buscará anomalías y valores faltantes en el conjunto de datos entrante con respecto al esquema.


### Cómo escribir un código de entrenamiento modelo

Necesitamos escribir el código del modelo como lo hicimos en el [Tutorial de canalizaciones simples de TFX](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

El modelo en sí es el mismo que en el tutorial anterior, pero esta vez usaremos el esquema generado en la canalización anterior en lugar de especificar características manualmente. La mayor parte del código se mantuvo intacta. La única diferencia es que no necesitamos especificar los nombres y tipos de funciones en este archivo. En cambio, los leemos del archivo de *esquema*.

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_metadata.proto.v0 import schema_pb2

# We don't need to specify _FEATURE_KEYS and _FEATURE_SPEC any more.
# Those information can be read from the given schema file.

_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model(schema: schema_pb2.Schema) -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.

  # ++ Changed code: Uses all features in the schema except the label.
  feature_keys = [f.name for f in schema.feature if f.name != _LABEL_KEY]
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in feature_keys]
  # ++ End of the changed code.

  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # ++ Changed code: Reads in schema file passed to the Trainer component.
  schema = tfx.utils.parse_pbtxt_file(fn_args.schema_path, schema_pb2.Schema())
  # ++ End of the changed code.

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model(schema)
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

Ahora ha completado todos los pasos de preparación para compilar una canalización de TFX para entrenamiento de modelos.

### Cómo escribir una definición de canalización

Agregaremos dos nuevos componentes, `Importer` y `ExampleValidator`. Importer trae un archivo externo a la canalización de TFX. En este caso, es un archivo que contiene la definición del esquema. ExampleValidator examinará los datos de entrada y validará si todos los datos de entrada se ajustan al esquema de datos que proporcionamos.


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     schema_path: str, module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a pipeline using predefined schema with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Computes statistics over data for visualization and example validation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # NEW: Import the schema.
  schema_importer = tfx.dsl.Importer(
      source_uri=schema_path,
      artifact_type=tfx.types.standard_artifacts.Schema).with_id(
          'schema_importer')

  # NEW: Performs anomaly detection based on statistics and data schema.
  example_validator = tfx.components.ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_importer.outputs['result'])

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      schema=schema_importer.outputs['result'],  # Pass the imported schema.
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,

      # NEW: Following three components were added to the pipeline.
      statistics_gen,
      schema_importer,
      example_validator,

      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

### Cómo ejecutar la canalización


In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      schema_path=SCHEMA_PATH,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

Debería ver el mensaje "INFO:absl:Component Pusher is finished" (INFO:absl:Component Pusher finalizó) si la canalización se completó correctamente.

### Análisis de salidas de la canalización

Hemos entrenado el modelo de clasificación de pingüinos y también hemos validado los ejemplos de entrada en el componente ExampleValidator. Podemos analizar la salida de ExampleValidator como lo hicimos con la canalización anterior.

In [ ]:
metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:
  ev_output = get_latest_artifacts(metadata_handler, PIPELINE_NAME,
                                   'ExampleValidator')
  anomalies_artifacts = ev_output[standard_component_specs.ANOMALIES_KEY]

También se pueden visualizar ExampleAnomalies desde ExampleValidator.

In [ ]:
visualize_artifacts(anomalies_artifacts)

Debería ver el mensaje "No anomalies found" (No se encontraron anomalías) para cada división de ejemplos. Debido a que utilizamos los mismos datos que se utilizaron para la generación del esquema en esta canalización, no se espera ninguna anomalía en este punto. Si ejecuta esta canalización repetidamente con nuevos datos entrantes, ExampleValidator debería poder encontrar cualquier discrepancia entre los nuevos datos y el esquema existente.

Si se encuentra alguna anomalía, puede revisar sus datos para comprobar si algún ejemplo no sigue sus suposiciones. Las salidas de otros componentes como StatisticsGen pueden resultar útiles. Sin embargo, cualquier anomalía que se encuentre NO bloqueará futuras ejecuciones de la canalización.

## Siguientes pasos

Puede encontrar más recursos en https://www.tensorflow.org/tfx/tutorials.

Consulte [Explicación de las canalizaciones de TFX](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines) para obtener más información sobre distintos conceptos en TFX.
